# Imports

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!export PYSPARK_PYTHON=python
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math
from datetime import datetime

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Inicialización del RDD

In [5]:
id2="1DBCzFlJWzdVirjQmoyTaH3GekLTzrznm"
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('googleplaystore_user_reviews.csv')

In [6]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)

rev_df = sqlContext.read.csv('googleplaystore_user_reviews.csv',
                              header=True,
                              inferSchema=True) \
                            .select("App", "Sentiment_Polarity")
rev_rdd = rev_df.rdd

# Pregunta #8
[*ir al notebook general*](https://colab.research.google.com/drive/1wWj79hKJTtaauQYJndtlhfQ8KEiuKe8Q?ouid=101857797731019568886&usp=drive_link)

> *Devolver la aplicación más polarizante. tener en cuenta el promedio de polarización de las reviews de cada aplicación.*

**Respuesta:** Homework (promedio de 1.0)

**Notas:**
- Medio rara la respuesta, debe ser de esas que tienen 1 review
- Soy de la opinión de que debería tomar el promedio del *valor absoluto* de las polaridades, pero [dijeron en slack](https://organizacinde-ea47838.slack.com/archives/C02DN3BCUHJ/p1695920493443789) que como era un ejercicio de 1 estrella no hacía falta complicarse la vida (igual me fijé y el resultado no cambia)
- Lo corrí con `%%timeit` *bastantes veces* y la performance usando `reduce()` y usando `max()` es *prácticamente igual*, pero me dijeron que use reduce y aprendí en algo3 que es mejor la nota que ser feliz (?
- La función `to_float()` es un poco bruta para mi gusto pero es sencilla y hace exactamente lo que quiero (pasarme las cosas a float y volar todo lo que no sea un float)

In [7]:
def to_float(str):
  try:
    return float(str)
  except:
    return math.nan

rev_rdd \
  .map(lambda x: (x[0], (to_float(x[1]), 1))) \
  .filter(lambda x: not math.isnan(x[1][0])) \
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
  .map(lambda x: (x[0], x[1][0]/x[1][1])) \
  .reduce(lambda x,y: x if x[1]>y[1] else y)

('HomeWork', 1.0)